# Tune Tutorial

<img src="tune.png" alt="Tune Logo" width="400"/>


Tune is a scalable framework for model training and hyperparameter search with a focus on deep learning and deep reinforcement learning.

**Code**: https://github.com/ray-project/ray/tree/master/python/ray/tune

**Examples**: https://github.com/ray-project/ray/tree/master/python/ray/tune/examples

**Documentation**: http://ray.readthedocs.io/en/latest/tune.html

**Mailing List** https://groups.google.com/forum/#!forum/ray-dev

# Overview

Tuning hyperparameters is often the most expensive part of the machine learning workflow. Tune is built to address this, demonstrating an efficient and scalable solution for this pain point.


## Outline
This tutorial will walk you through the following process:

1. Creating and training a model on a toy dataset (MNIST)
2. Integrating Tune into your workflow
3. Trying out advanced features - plugging in an efficient scheduler
4. Validating your trained model
5. (Optional) Try out a search algorithm


In [35]:
from helper import *
import numpy as np
from IPython.display import HTML
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

limit_threads(4)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## PART 1: Creating a model to be trained.

Let's create a Convolutional Neural Network model. Convolutional Neural Networks (ConvNets or CNNs) are a category of Neural Networks that have proven very effective in areas such as image recognition and classification. The details of how a Convolutional Neural Network works are unimportant here, but you're welcome to read more about them here: http://cs231n.github.io/convolutional-networks/

<img src="cnn.png" alt="MNIST Visualization" width="800"/>


This convolutional neural network model will be used classify digits (from the MNIST dataset).

<img src="mnist.png" alt="MNIST Visualization" width="400"/>

This is a fairly simple dataset, but it enables us to explore Tune's functionality in depth.
We will use 60,000 images to train the network. The images are 28x28 NumPy arrays, with pixel values ranging between 0 and 255. The labels are an array of integers, ranging from 0 to 9. These correspond to the digit the image represents.

Here, we'll specify some arguments and some reasonable defaults for this model. These are the hyperparameters settings that we will later use to further optimize this model.

In [2]:
import argparse
parser = argparse.ArgumentParser(description='Keras MNIST Example')
parser.add_argument('--lr', type=float, default=0.1, help='learning rate')
parser.add_argument('--momentum', type=float, default=0.0, help='SGD momentum')
parser.add_argument('--kernel1', type=int, default=3, help='Size of first kernel')
parser.add_argument('--kernel2', type=int, default=3, help='Size of second kernel')
parser.add_argument('--poolsize', type=int, default=2, help='Size of Poolin')
parser.add_argument('--dropout1', type=float, default=0.25, help='Size of first kernel')
parser.add_argument('--hidden', type=int, default=8, help='Size of Hidden Layer')
parser.add_argument('--dropout2', type=float, default=0.5, help='Size of first kernel')

DEFAULT_ARGS = vars(parser.parse_known_args()[0])

This below function will create and return a Convolutional Neural Network. You don't need to modify this function

In [61]:
def make_model(parameters):
    config = DEFAULT_ARGS.copy()  # This is obtained via the global scope
    config.update(parameters)
    num_classes = 10
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(config["kernel1"], config["kernel1"]),
                     activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (config["kernel2"], config["kernel2"]), activation='relu'))
    model.add(MaxPooling2D(pool_size=(config["poolsize"], config["poolsize"])))
    model.add(Dropout(config["dropout1"]))
    model.add(Flatten())
    model.add(Dense(config["hidden"], activation='relu'))
    model.add(Dropout(config["dropout2"]))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(
                      lr=config["lr"], momentum=config["momentum"]),
                  metrics=['accuracy'])
    return model

### Exercise: Setup a basic model training script.

The process of training the neural network model occurs as follows:

1. Feed the training data to the model—in this example, the `batch_of_data` and `batch_of_labels` arrays.
2. The model learns to associate images and labels.

**Exercise**: Finish the TODOs below. Here are a few hints:

1) `data_generator` is an iterator that returns (`batch_of_data`, `batch_of_labels`), like follows:

```python
for batch_of_data, batch_of_labels in data_generator:
    do_something_interesting()
```
2) You can use `model.fit(batch_of_data, batch_of_labels)` to repeatedly improve the model.

In [62]:
def train_mnist(args):
    """Loads data, does one pass over the data, and saves the weights."""
    data_generator = load_data()
    model = make_model(args)
    for batch_of_data, batch_of_labels in data_generator:
        model.fit(batch_of_data, batch_of_labels)
    ## TODO: use the `data_generator` to iterate over the data and improve the model
    model.save_weights("./weights.h5")
    return model

Let's run this above training script to make sure things work.

In [66]:
first_model = train_mnist(DEFAULT_ARGS)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/1
32/32 [==============================] - 1s 16ms/step - loss: 2.3299 - acc: 0.0312
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2972 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.3076 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2745 - acc: 0.1250
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2954 - acc: 0.0625
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2513 - acc: 0.1875
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2324 - acc: 0.2188
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.2004 - acc: 0.1562
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.3327 - acc: 0.0938
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 2.1915 - acc: 0.3438
Epoch 

Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.3826 - acc: 0.4375
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.5367 - acc: 0.3438
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.7033 - acc: 0.4375
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.4239 - acc: 0.4375
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.4703 - acc: 0.4688
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.6093 - acc: 0.2812
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.4112 - acc: 0.5000
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.3055 - acc: 0.4375
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.6422 - acc: 0.2500
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.3539 - acc: 0.4688
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.3128 - 

32/32 [==============================] - 0s 1ms/step - loss: 1.0907 - acc: 0.5938
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.0847 - acc: 0.5938
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 0.7445 - acc: 0.8125
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 0.9382 - acc: 0.6562
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.4749 - acc: 0.5312
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.1482 - acc: 0.5938
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.3060 - acc: 0.5000
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.7142 - acc: 0.4688
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.7678 - acc: 0.3125
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.4383 - acc: 0.5000
Epoch 1/1
32/32 [==============================] - 0s 1ms/step - loss: 1.4435 - acc: 0.406

In [68]:
evaluate(first_model)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
10000/10000 [==============================] - 4s 423us/step
Model evaluation results: {'loss': 2.2909044315338134, 'acc': 0.1375}


Let's now quickly try out this model to see if it works as expected. We'll load the model with our trained weights.

In [64]:
#first_model = make_model(DEFAULT_ARGS)
#first_model.load_weights("./weights.h5")

**Exercise**: Try to write a digit into the box below. This will automatically save your input in a variable `data` behind the scenes.

In [7]:
data = None
HTML(open("input.html").read())

(tip: don't expect it to work)

In [8]:
prepared_data = prepare_data(data)
print("This model predicted your input as", first_model.predict(prepared_data).argmax())

NameError: name 'data' is not defined

**You've now set up a model that we can use Tune to optimize!**

## Part 2: Setting up Tune

One thing we might want to do now is find better hyperparameters so that our model trains more quickly and possibly to a higher accuracy. Let's make some minor modifications to utilize Tune. 

Tune uses Ray as a backend, so we will first import and initialize Ray.

In [9]:
import ray
from ray import tune

ray.init(ignore_reinit_error=True)

Detected environment variable 'RAY_USE_XRAY'.
Process STDOUT and STDERR is being redirected to /tmp/ray/session_2018-10-06_17-39-51_43964/logs.
Waiting for redis server at 127.0.0.1:36457 to respond...
Waiting for redis server at 127.0.0.1:12600 to respond...
Starting the Plasma object store with 108.00 GB memory.

View the web UI at http://localhost:8888/notebooks/ray_ui.ipynb?token=209787b0b9203ca2ef4d20146b144eb4ec4c9cb0d0638793



{'node_ip_address': '169.229.49.174',
 'redis_address': '169.229.49.174:36457',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/ray/session_2018-10-06_17-39-51_43964/sockets/plasma_store', manager_name=None, manager_port=None)],
 'local_scheduler_socket_names': [],
 'raylet_socket_names': ['/tmp/ray/session_2018-10-06_17-39-51_43964/sockets/raylet'],
 'webui_url': 'http://localhost:8888/notebooks/ray_ui.ipynb?token=209787b0b9203ca2ef4d20146b144eb4ec4c9cb0d0638793'}

Tune will automate and distribute your hyperparameter search by scheduling a number of **trials** on a machine. Each trial runs a user-defined Python function with a sampled set of hyperparameters. 

### Exercise: Two steps to use Tune:

Step 1) For the function you wish to tune, we need to change the signature to a specific format as shown below. Specifically: **pass in a ``reporter`` object to the below `train_mnist_tune` class**.

```python
def trainable(config, reporter):
    """
    Args:
        config (dict): Parameters provided from the search algorithm
            or variant generation.
        reporter (Reporter): Handle to report intermediate metrics to Tune.
    """
```

Step 2) We want to keep track of performance as the model is training. Specifically: **get the `mean_accuracy` from Keras, and call the ``reporter`` to report the `mean_accuracy` for every batch**. 

You can get model accuracy from Keras with the following code:

```python
result = model.fit(x_batch, y_batch, verbose=0)
mean_accuracy = result.history["acc"][0]
```


Example of using the reporter:

```python
def train_func(config, reporter):  # add a reporter arg
    # ...
    for data, target in dataset:
        result = model.fit(data, target)
        accuracy = process_result(result)
        reporter(mean_accuracy=accuracy) # report metrics
```


In [24]:
def train_mnist_tune(config, reporter): ### TODO: Change this function signature following step 1 #####
    data_generator = load_data()
    model = make_model(config)
    for i, (x_batch, y_batch) in enumerate(data_generator):
        result = model.fit(x_batch, y_batch, verbose=0)
        reporter(mean_accuracy=result.history["acc"][0])
        ### TODO: Use the reporter here to fill out intermediate metrics following step 2###
        ### Don't change the below 
        if i % 2 == 0:
            model.save_weights("./weights_tune.h5")

In [25]:
# This may take 30 seconds or so to run if incorrectly written
assert test_reporter(train_mnist_tune)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Works!


**If you've done this correctly, you now have properly converted your function to be Tune-compatible!**

### Exercise: Let's now try to search over some parameters. 

*NOTE: You can find the documentation for this section here: https://ray.readthedocs.io/en/latest/tune-usage.html#specifying-experiments*

In this section, we'll use some basic Tune features for training - namely specifying a stopping criteria and a search space. 

Let's first create a Tune Experiment specification. The relevant documentation for the Experiment class is here:

```python
class ray.tune.Experiment(name, run, stop=None, config=None, ... ):
    """Tracks experiment specifications.

    Parameters:
        name (str): Name of experiment.
        run (function|class|str): The algorithm or model to train.
            This may refer to the name of a built-on algorithm
            (e.g. RLLib's DQN or PPO), a user-defined trainable
            function or class, or the string identifier of a
            trainable function or class registered in the tune registry.
        stop (dict): The stopping criteria. The keys may be any field in
            the return result of 'train()', whichever is reached first.
            Defaults to empty dict.
        config (dict): Algorithm-specific configuration for Tune variant
            generation (e.g. env, hyperparams). Defaults to empty dict.
            Custom search algorithms may ignore this.
        trial_resources (dict): Machine resources to allocate per trial,
            e.g. ``{"cpu": 64, "gpu": 8}``. Note that GPUs will not be
            assigned unless you specify them here. Defaults to 1 CPU and 0
            GPUs in ``Trainable.default_resource_request()``.
        ...
```

**Part 1**: First, **set the stopping criteria to when `mean_accuracy` passes `0.95`**. For example, to specify that trials will be stopped whenever they report `arbitrary_metric` that is `>= 500`, do:

```python
stop={"arbitrary_metric": 500}
```


**Part 2**: We also want to designate a search space. We'll search over *learning rate*, which sets the step size of our model update, and *momentum*, which helps accelerate gradients vectors in the right directions, thus leading to faster converging.

You can use `tune.grid_search` to specify an axis of a grid search. By default, Tune also supports sampling parameters from user-specified lambda functions, which can be used independently or in combination with grid search.  The following example shows grid search over a set of values combined with random sampling from a lambda functions, generating 3 different trials. 

```python
configuration = tune.Experiment(
    # ...
    config={
        "arbitrary_parameter1": lambda spec: np.random.uniform(0.1, 100),
        "arbitrary_parameter2": tune.grid_search([16, 64, 256]),
        # ...
    }
)
```

Specifically, 
1. randomly search for learning rate `"lr"` between 0.001 to 0.1,
2. do a grid search over `"momentum"` for `[0.2, 0.4, 0.6]` 

In [26]:
configuration = tune.Experiment(
    "experiment_name",
    run=train_mnist_tune,
    trial_resources={"cpu": 4},
    stop={"mean_accuracy": 0.95},  # TODO: Part 1
    config={"lr": lambda spec: np.random.uniform(0.001, 0.1),
            "momentum": tune.grid_search([0.2, 0.4, 0.6])}  # TODO: Part 2
)

assert configuration.spec.get("stop", {}).get("mean_accuracy") == 0.95
assert "grid_search" in configuration.spec.get("config", {}).get("momentum", {})
assert "lr" in configuration.spec.get("config", {})
print("Success!")

Success!


Now, we can run our experiment with a single line of code. 

*Note*: Be sure pay attention to the `acc` metric next to each running trial. That indicates the most recently reported mean accuracy for that trial. This should evaluate in less than a minute. The output will look something similar to:

```
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 8/8 CPUs, 0/1 GPUs
Result logdir: .../ray_results/experiment_name
RUNNING trials:
 - train_mnist_tune_0_lr=0.085836,momentum=0.2:	RUNNING [pid=44320], 4 s, 3 iter, 0.406 acc
 - train_mnist_tune_1_lr=0.062562,momentum=0.4:	RUNNING [pid=44321], 3 s, 2 iter, 0.219 acc
 - train_mnist_tune_2_lr=0.099461,momentum=0.6:	RUNNING [pid=44317], 3 s, 2 iter, 0.281 acc
 ```

In [27]:
import sys
# sys.stdout
trials = tune.run_experiments(configuration, verbose=False)

== Status ==
Using FIFO scheduling algorithm.


Created LogSyncer for /home/eecs/rliaw/ray_results/experiment_name/train_mnist_tune_0_lr=0.085836,momentum=0.2_2018-10-06_17-43-156a6h592n -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment_name
PENDING trials:
 - train_mnist_tune_1_lr=0.062562,momentum=0.4:	PENDING
 - train_mnist_tune_2_lr=0.099461,momentum=0.6:	PENDING
RUNNING trials:
 - train_mnist_tune_0_lr=0.085836,momentum=0.2:	RUNNING

Created LogSyncer for /home/eecs/rliaw/ray_results/experiment_name/train_mnist_tune_1_lr=0.062562,momentum=0.4_2018-10-06_17-43-15aw4iq6ag -> 
Created LogSyncer for /home/eecs/rliaw/ray_results/experiment_name/train_mnist_tune_2_lr=0.099461,momentum=0.6_2018-10-06_17-43-15cnvcddsn -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 12/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment_name
RUNNING trials:
 - train_mn

You can expect the result below to be about `0.6`, although your mileage may vary (and it's OK).

In [28]:
print("The best result is", get_best_result(trials, metric="mean_accuracy"))

The best result is {'mean_accuracy': 0.6875}


**You've run your first Tune experiment!**

## Exercise: Try using a scheduler



By default, Tune schedules trials in serial order with the `FIFOScheduler` class. However, you can also specify a custom scheduling algorithm that can early stop trials or perturb parameters. 

Let's use a state of the art algorithm, `HyperBand`, to scale up and accelerate our training. Hyperband is an algorithm that focuses on speeding up random search through adaptive resource allocation and early-stopping.

**Part 1**: Let's scale up our search. 

1) Sample the search space 10 times. (https://ray.readthedocs.io/en/latest/tune-usage.html#sampling-multiple-times).

2) Search over another hyperparameter: `"hidden"` from 16 to 512 which specifies the size of the last neural network layer.

Here, use `np.random.randint`. 
```python
numpy.random.randint(low, high=None, size=None, dtype='l')
    """Return random integers from low (inclusive) to high (exclusive)."""
```

An extended version of the `Experiment` documentation is shown below. Note that you should expect a total of 30 trials, due to the usage of `grid_search`.

```python
class ray.tune.Experiment(name, run, stop=None, config=None, ... ):
    """Tracks experiment specifications.

    Parameters:
        name (str): Name of experiment.
        run (function|class|str): The algorithm or model to train.
            This may refer to the name of a built-on algorithm
            (e.g. RLLib's DQN or PPO), a user-defined trainable
            function or class, or the string identifier of a
            trainable function or class registered in the tune registry.
        stop (dict): The stopping criteria. The keys may be any field in
            the return result of 'train()', whichever is reached first.
            Defaults to empty dict.
        config (dict): Algorithm-specific configuration for Tune variant
            generation (e.g. env, hyperparams). Defaults to empty dict.
            Custom search algorithms may ignore this.
        trial_resources (dict): Machine resources to allocate per trial,
            e.g. ``{"cpu": 64, "gpu": 8}``. Note that GPUs will not be
            assigned unless you specify them here. Defaults to 1 CPU and 0
            GPUs in ``Trainable.default_resource_request()``.
        num_samples (int): Number of times to sample from the
            hyperparameter space. Defaults to 1. If `grid_search` is
            provided as an argument, the grid will be repeated
            `num_samples` of times.
        ...
```

In [29]:
configuration2 = tune.Experiment(
    "experiment2",
    run=train_mnist_tune,
    num_samples=10, ## TODO: Change this to 10
    trial_resources={"cpu": 4},
    stop={"mean_accuracy": 0.95},
    config={
        "lr": lambda spec: np.random.uniform(0.001, 0.1),
        "momentum": tune.grid_search([0.2, 0.4, 0.6]),
        "hidden": lambda spec: np.random.randint(16, high=513), ## TODO: Sample uniformly from 16 to 512
    }
)


**Part 2**: Create an Asynchronous HyperBand Scheduler (https://ray.readthedocs.io/en/latest/tune-schedulers.html#asynchronous-hyperband). The documentation is shown below. 

Be sure to set the `time_attr` to `training_iteration` and `reward_attr` to `mean_accuracy`.

```python
class AsyncHyperBandScheduler(FIFOScheduler):
    """This provides HyperBand functionality to your search.
    
    Hyperband is an algorithm that focuses on speeding up 
    random search through adaptive resource allocation and early-stopping.

    See https://openreview.net/forum?id=S1Y7OOlRZ

    Args:
        time_attr (str): A training result attr to use for comparing time.
            Note that you can pass in something non-temporal such as
            `training_iteration` as a measure of progress, the only requirement
            is that the attribute should increase monotonically.
        reward_attr (str): The training result objective value attribute. As
            with `time_attr`, this may refer to any objective value. Stopping
            procedures will use this attribute.
        ...
        
    Examples:
        >>> hyperband = AsyncHyperBandScheduler(
        >>>     time_attr='training_iteration',
        >>>     reward_attr='mean_accuracy')
    """

```

In [30]:
from ray.tune.schedulers import AsyncHyperBandScheduler

## TODO: Create an Asynchronous HyperBand Scheduler
hyperband = AsyncHyperBandScheduler(
    time_attr='training_iteration',
    reward_attr='mean_accuracy')

Given the previous configuration, pass in the HyperBand scheduler to `run_experiments`.

Recall that the `run_experiments` API is:
```python
def run_experiments(experiments=None,
                    search_alg=None,
                    scheduler=None,
                    ...):
    """Runs and blocks until all trials finish.

    Args:
        experiments (Experiment | list | dict): Experiments to run. Will be
            passed to `search_alg` via `add_configurations`.
        search_alg (SearchAlgorithm): Search Algorithm. Defaults to
            BasicVariantGenerator.
        scheduler (TrialScheduler): Scheduler for executing
            the experiment. Choose among FIFO (default), MedianStopping,
            AsyncHyperBand, and HyperBand.
        ...
    
    Returns:
        List of Trial objects, holding data for each executed trial.
```


**Exercise**: Call `run_experiments` with the correct arguments. This may take multiple minutes for the experiment to complete.

In [31]:
# Note that you should be using `configuration2` instead of `configuration`.
trials = tune.run_experiments(configuration2, scheduler=hyperband, verbose=False)

== Status ==
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 90.000: None | Iter 30.000: None | Iter 10.000: None
Bracket: Iter 90.000: None | Iter 30.000: None
Bracket: Iter 90.000: None


Created LogSyncer for /home/eecs/rliaw/ray_results/experiment2/train_mnist_tune_0_hidden=466,lr=0.016227,momentum=0.2_2018-10-06_17-44-1042c790pm -> 
== Status ==
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 90.000: None | Iter 30.000: None | Iter 10.000: None
Bracket: Iter 90.000: None | Iter 30.000: None
Bracket: Iter 90.000: None
Resources requested: 4/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
PENDING trials:
 - train_mnist_tune_1_hidden=281,lr=0.057856,momentum=0.4:	PENDING
 - train_mnist_tune_2_hidden=453,lr=0.07187,momentum=0.6:	PENDING
 - train_mnist_tune_3_hidden=285,lr=0.093658,momentum=0.2:	PENDING
 - train_mnist_tune_4_hidden=418,lr=0.0039973,momentum=0.4:	PENDING
 - train_mnist_tune_5_hidden=47,lr=0.01716,momentum=0.6:	PENDING
 - train_mnist_tune_6_

== Status ==
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 90.000: None | Iter 30.000: None | Iter 10.000: None
Bracket: Iter 90.000: None | Iter 30.000: None
Bracket: Iter 90.000: None
Resources requested: 48/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
PENDING trials:
 - train_mnist_tune_12_hidden=166,lr=0.062701,momentum=0.2:	PENDING
 - train_mnist_tune_13_hidden=493,lr=0.026987,momentum=0.4:	PENDING
 - train_mnist_tune_14_hidden=499,lr=0.058082,momentum=0.6:	PENDING
 - train_mnist_tune_15_hidden=479,lr=0.037516,momentum=0.2:	PENDING
 - train_mnist_tune_16_hidden=219,lr=0.031714,momentum=0.4:	PENDING
  ... 8 not shown
 - train_mnist_tune_25_hidden=100,lr=0.028686,momentum=0.4:	PENDING
 - train_mnist_tune_26_hidden=50,lr=0.006687,momentum=0.6:	PENDING
 - train_mnist_tune_27_hidden=473,lr=0.0013436,momentum=0.2:	PENDING
 - train_mnist_tune_28_hidden=385,lr=0.056879,momentum=0.4:	PENDING
 - train_mnist_tune_29_hidden=409,lr=0.022294,momentum=0.6:	PENDIN

Created LogSyncer for /home/eecs/rliaw/ray_results/experiment2/train_mnist_tune_12_hidden=166,lr=0.062701,momentum=0.2_2018-10-06_17-44-52batxk6ku -> 
== Status ==
Using AsyncHyperBand: num_stopped=1
Bracket: Iter 90.000: None | Iter 30.000: None | Iter 10.000: 0.30208333333333337
Bracket: Iter 90.000: None | Iter 30.000: None
Bracket: Iter 90.000: None
Resources requested: 48/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
PENDING trials:
 - train_mnist_tune_13_hidden=493,lr=0.026987,momentum=0.4:	PENDING
 - train_mnist_tune_14_hidden=499,lr=0.058082,momentum=0.6:	PENDING
 - train_mnist_tune_15_hidden=479,lr=0.037516,momentum=0.2:	PENDING
 - train_mnist_tune_16_hidden=219,lr=0.031714,momentum=0.4:	PENDING
 - train_mnist_tune_17_hidden=222,lr=0.081578,momentum=0.6:	PENDING
  ... 7 not shown
 - train_mnist_tune_25_hidden=100,lr=0.028686,momentum=0.4:	PENDING
 - train_mnist_tune_26_hidden=50,lr=0.006687,momentum=0.6:	PENDING
 - train_mnist_tune_27_hidden=473,lr=

== Status ==
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 90.000: None | Iter 30.000: None | Iter 10.000: 0.5416666666666667
Bracket: Iter 90.000: None | Iter 30.000: None
Bracket: Iter 90.000: None
Resources requested: 48/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
PENDING trials:
 - train_mnist_tune_14_hidden=499,lr=0.058082,momentum=0.6:	PENDING
 - train_mnist_tune_15_hidden=479,lr=0.037516,momentum=0.2:	PENDING
 - train_mnist_tune_16_hidden=219,lr=0.031714,momentum=0.4:	PENDING
 - train_mnist_tune_17_hidden=222,lr=0.081578,momentum=0.6:	PENDING
  ... 8 not shown
 - train_mnist_tune_26_hidden=50,lr=0.006687,momentum=0.6:	PENDING
 - train_mnist_tune_27_hidden=473,lr=0.0013436,momentum=0.2:	PENDING
 - train_mnist_tune_28_hidden=385,lr=0.056879,momentum=0.4:	PENDING
 - train_mnist_tune_29_hidden=409,lr=0.022294,momentum=0.6:	PENDING
RUNNING trials:
 - train_mnist_tune_0_hidden=466,lr=0.016227,momentum=0.2:	RUNNING [pid=44313], 22 s, 17 iter, 0.469 acc

== Status ==
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 90.000: None | Iter 30.000: None | Iter 10.000: 0.5416666666666667
Bracket: Iter 90.000: None | Iter 30.000: None
Bracket: Iter 90.000: None
Resources requested: 48/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
PENDING trials:
 - train_mnist_tune_14_hidden=499,lr=0.058082,momentum=0.6:	PENDING
 - train_mnist_tune_15_hidden=479,lr=0.037516,momentum=0.2:	PENDING
 - train_mnist_tune_16_hidden=219,lr=0.031714,momentum=0.4:	PENDING
 - train_mnist_tune_17_hidden=222,lr=0.081578,momentum=0.6:	PENDING
  ... 8 not shown
 - train_mnist_tune_26_hidden=50,lr=0.006687,momentum=0.6:	PENDING
 - train_mnist_tune_27_hidden=473,lr=0.0013436,momentum=0.2:	PENDING
 - train_mnist_tune_28_hidden=385,lr=0.056879,momentum=0.4:	PENDING
 - train_mnist_tune_29_hidden=409,lr=0.022294,momentum=0.6:	PENDING
RUNNING trials:
 - train_mnist_tune_0_hidden=466,lr=0.016227,momentum=0.2:	RUNNING [pid=44313], 33 s, 25 iter, 0.594 acc

...

Created LogSyncer for /home/eecs/rliaw/ray_results/experiment2/train_mnist_tune_26_hidden=50,lr=0.006687,momentum=0.6_2018-10-06_17-47-4771lfajx7 -> 
== Status ==
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 90.000: None | Iter 30.000: 0.75 | Iter 10.000: 0.625
Bracket: Iter 90.000: None | Iter 30.000: 0.78125
Bracket: Iter 90.000: None
Resources requested: 48/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
PENDING trials:
 - train_mnist_tune_27_hidden=473,lr=0.0013436,momentum=0.2:	PENDING
 - train_mnist_tune_28_hidden=385,lr=0.056879,momentum=0.4:	PENDING
 - train_mnist_tune_29_hidden=409,lr=0.022294,momentum=0.6:	PENDING
RUNNING trials:
 - train_mnist_tune_0_hidden=466,lr=0.016227,momentum=0.2:	RUNNING [pid=44313], 81 s, 64 iter, 0.938 acc
 - train_mnist_tune_12_hidden=166,lr=0.062701,momentum=0.2:	RUNNING [pid=1867], 62 s, 48 iter, 0.75 acc
 - train_mnist_tune_14_hidden=499,lr=0.058082,momentum=0.6:	RUNNING [pid=1869], 41 s, 29 iter, 0.75 acc
 - train

Created LogSyncer for /home/eecs/rliaw/ray_results/experiment2/train_mnist_tune_28_hidden=385,lr=0.056879,momentum=0.4_2018-10-06_17-48-11rzh_f20m -> 
== Status ==
Using AsyncHyperBand: num_stopped=11
Bracket: Iter 90.000: None | Iter 30.000: 0.75 | Iter 10.000: 0.5833333333333334
Bracket: Iter 90.000: None | Iter 30.000: 0.78125
Bracket: Iter 90.000: None
Resources requested: 48/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
PENDING trials:
 - train_mnist_tune_29_hidden=409,lr=0.022294,momentum=0.6:	PENDING
RUNNING trials:
 - train_mnist_tune_0_hidden=466,lr=0.016227,momentum=0.2:	RUNNING [pid=44313], 91 s, 72 iter, 0.844 acc
 - train_mnist_tune_12_hidden=166,lr=0.062701,momentum=0.2:	RUNNING [pid=1867], 71 s, 56 iter, 0.875 acc
 - train_mnist_tune_14_hidden=499,lr=0.058082,momentum=0.6:	RUNNING [pid=1869], 51 s, 37 iter, 0.875 acc
 - train_mnist_tune_15_hidden=479,lr=0.037516,momentum=0.2:	RUNNING [pid=1876], 52 s, 36 iter, 0.625 acc
 - train_mnist_tune_16_

== Status ==
Using AsyncHyperBand: num_stopped=13
Bracket: Iter 90.000: None | Iter 30.000: 0.75 | Iter 10.000: 0.5833333333333334
Bracket: Iter 90.000: None | Iter 30.000: 0.78125
Bracket: Iter 90.000: None
Resources requested: 40/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
RUNNING trials:
 - train_mnist_tune_0_hidden=466,lr=0.016227,momentum=0.2:	RUNNING [pid=44313], 100 s, 80 iter, 0.906 acc
 - train_mnist_tune_14_hidden=499,lr=0.058082,momentum=0.6:	RUNNING [pid=1869], 58 s, 44 iter, 0.875 acc
 - train_mnist_tune_15_hidden=479,lr=0.037516,momentum=0.2:	RUNNING [pid=1876], 62 s, 44 iter, 0.938 acc
 - train_mnist_tune_16_hidden=219,lr=0.031714,momentum=0.4:	RUNNING [pid=1862], 58 s, 43 iter, 0.875 acc
 - train_mnist_tune_19_hidden=387,lr=0.074925,momentum=0.4:	RUNNING [pid=1860], 41 s, 29 iter, 0.812 acc
 - train_mnist_tune_21_hidden=372,lr=0.0061144,momentum=0.2:	RUNNING [pid=1262], 37 s, 25 iter, 0.281 acc
 - train_mnist_tune_25_hidden=100,lr=0.028686,

== Status ==
Using AsyncHyperBand: num_stopped=15
Bracket: Iter 90.000: None | Iter 30.000: 0.75 | Iter 10.000: 0.5833333333333334
Bracket: Iter 90.000: None | Iter 30.000: 0.78125
Bracket: Iter 90.000: None
Resources requested: 24/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
RUNNING trials:
 - train_mnist_tune_15_hidden=479,lr=0.037516,momentum=0.2:	RUNNING [pid=1876], 80 s, 59 iter, 0.906 acc
 - train_mnist_tune_16_hidden=219,lr=0.031714,momentum=0.4:	RUNNING [pid=1862], 73 s, 57 iter, 0.844 acc
 - train_mnist_tune_25_hidden=100,lr=0.028686,momentum=0.4:	RUNNING [pid=1263], 37 s, 26 iter, 0.812 acc
 - train_mnist_tune_27_hidden=473,lr=0.0013436,momentum=0.2:	RUNNING [pid=1271], 31 s, 20 iter, 0.125 acc
 - train_mnist_tune_28_hidden=385,lr=0.056879,momentum=0.4:	RUNNING [pid=1264], 26 s, 16 iter, 0.594 acc
 - train_mnist_tune_29_hidden=409,lr=0.022294,momentum=0.6:	RUNNING [pid=1270], 25 s, 14 iter, 0.469 acc
TERMINATED trials:
 - train_mnist_tune_0_hidden

== Status ==
Using AsyncHyperBand: num_stopped=18
Bracket: Iter 90.000: None | Iter 30.000: 0.7604166666666667 | Iter 10.000: 0.5833333333333334
Bracket: Iter 90.000: None | Iter 30.000: 0.7395833333333334
Bracket: Iter 90.000: None
Resources requested: 4/48 CPUs, 0/1 GPUs
Result logdir: /home/eecs/rliaw/ray_results/experiment2
RUNNING trials:
 - train_mnist_tune_25_hidden=100,lr=0.028686,momentum=0.4:	RUNNING [pid=1263], 56 s, 44 iter, 0.719 acc
TERMINATED trials:
 - train_mnist_tune_0_hidden=466,lr=0.016227,momentum=0.2:	TERMINATED [pid=44313], 105 s, 84 iter, 0.844 acc
 - train_mnist_tune_1_hidden=281,lr=0.057856,momentum=0.4:	TERMINATED [pid=44301], 73 s, 58 iter, 0.969 acc
 - train_mnist_tune_2_hidden=453,lr=0.07187,momentum=0.6:	TERMINATED [pid=44284], 68 s, 54 iter, 1 acc
 - train_mnist_tune_3_hidden=285,lr=0.093658,momentum=0.2:	TERMINATED [pid=44312], 56 s, 45 iter, 0.969 acc
 - train_mnist_tune_4_hidden=418,lr=0.0039973,momentum=0.4:	TERMINATED [pid=44306], 12 s, 10 iter, 0.2

**Exercise**: Now, let's get the best model from your search process, and check its validation accuracy compared to the first model we created above.

In [37]:
best_model = get_best_model(make_model, trials, metric="mean_accuracy")

Loading from /home/eecs/rliaw/ray_results/experiment2/train_mnist_tune_2_hidden=453,lr=0.07187,momentum=0.6_2018-10-06_17-44-10qurqr4yb/weights_tune.h5


In [65]:
print("Best model...")
evaluate(best_model)
print("First model...")
evaluate(first_model)

Best model...
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
10000/10000 [==============================] - 5s 534us/step
Model evaluation results: {'loss': 0.26608398395180705, 'acc': 0.9255}
First model...
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
10000/10000 [==============================] - 4s 401us/step
Model evaluation results: {'loss': 0.6691122355461121, 'acc': 0.8542}


## Try out your model on some manual inputs!

**Exercise**: write a digit into the canvas below. This will automatically load your input into the variable `final_data`. (Due to randomness, your mileage may vary. If your model is bad, try increasing the number of samples, or try the Optional Search Algorithm section below).

In [44]:
HTML(open("input_final.html").read())

In [60]:
manual_input = prepare_data(final_data)
best = best_model.predict(manual_input).argmax()
first = first_model.predict(manual_input).argmax()

print("Best model got {}, first model got {}".format(best, first))

Best model got 3, first model got 1


# Congratulations, you're now a Tune expert!

# Please: fill out this form to provide feedback on this tutorial!

https://goo.gl/forms/NVTFjUKFz4TH8kgK2

# (Optional) Try using a search algorithm

Tune is an execution layer, so we can combine powerful optimizers such as HyperOpt (https://github.com/hyperopt/hyperopt) with state-of-the-art algorithms such as HyperBand without modifying any model training code.

The documentation to doing this is here: https://ray.readthedocs.io/en/latest/tune-searchalg.html#hyperopt-search-tree-structured-parzen-estimators

In [69]:
from hyperopt import hp
from ray.tune.suggest import HyperOptSearch

space = {
    "lr": hp.uniform("lr", 0.001, 0.1),
    "momentum": hp.uniform("momentum", 0.1, 0.9),
    "hidden": hp.choice("hidden", np.arange(16, 256, dtype=int)),
    "dropout1": hp.uniform("dropout1", 0.2, 0.8),
}

## TODO: CREATE A HyperOptObject
hyperopt_search = HyperOptSearch(space, reward_attr="mean_accuracy")

## TODO: Pass in the object to Tune.
good_results = tune.run_experiments(
    configuration2, search_alg=hyperopt_search, scheduler=hyperband, verbose=False)

ModuleNotFoundError: No module named 'hyperopt'

In [ ]:
# Feel free to compare your best model here
best_model2 = get_best_model(make_model, trials, metric="mean_accuracy")

In [ ]:
# Using the canvas data from above 

manual_input = prepare_data(final_data)
best = best_model.predict(manual_input).argmax()
best2 = best_model2.predict(manual_input).argmax()
first = first_model.predict(manual_input).argmax()

print("Best model got {}, HyperOpt model got {}, first model got {}".format(best, best2, first))
